In [ ]:
!pip install -U seaborn
!pip install -U numpy 

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils import *
from label_utils import *

%matplotlib inline

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

### Clean and process original dataset for longitudinal analysis

In [ ]:
df = read_csv('path/to/file1.csv')

df = remove_not_consented_participants(df)
df = remove_consent_withdrawals(df)

fields_to_aggregate = [102, 4079, 4080] # pulse, dias, sys 
df = aggregate_repeat_measurements(df, field_ids=fields_to_aggregate, visit_id=0, drop_repeats_cols=True)
df = aggregate_repeat_measurements(df, field_ids=fields_to_aggregate, visit_id=1, drop_repeats_cols=True)
df = aggregate_repeat_measurements(df, field_ids=fields_to_aggregate, visit_id=2, drop_repeats_cols=True)
df = aggregate_repeat_measurements(df, field_ids=fields_to_aggregate, visit_id=3, drop_repeats_cols=True)

df = aggregate_visits_for_static_col(df, field_id=21000, method="most_common", drop_cols=True) # ethnicity (3 instances)
df = aggregate_visits_for_static_col(df, field_id=40007, method="min", drop_cols=True) # age at death (2 instances)
df = set_type_as_datetime(df, field_id=40000)
df = aggregate_visits_for_static_col(df, field_id=40000, method="min", drop_cols=True) # date of death (2 instances)

df = compute_age_at_visit(df, drop_col=True)
df = group_ethnicity_codes(df)

df.head()

In [ ]:
icd10_codes_selected = ["C18", "C19", "C20"]  # colorectal cancer codes
df_label = generate_labels_from_cancer_registry(df, icd10_codes_selected, mode="all_visits")
# df_label.to_csv("s3://ukb-colorectal-cancer/ukb_crc_longitudinal_v2_label.csv", index=False)
df_label.head()

In [ ]:
df_merged = pd.merge(df, df_label, how="left", on="eid")

# Remove unused cols
colnames = df_merged.columns
collection_time_cols = [col for col in colnames if col.startswith(("3166-", "20035-"))]
cancer_registry_cols = [col for col in colnames if col.startswith(("40009-", "40008-", "40005-", "40006-", "40011-", "40012-", "40013-", "40019-", "40021-"))] 
cancer_medhis_cols = [col for col in colnames if col.startswith(("84-", "134-", "20012-", "20001-"))]
death_registry_cols = [col for col in colnames if col.startswith(("40000-", "40001-", "40002-", "40007-", "40010-", "40018-", "40020-", "40023-"))] 
cols_keep = []
cols_to_drop = [col for col in collection_time_cols + cancer_registry_cols + cancer_medhis_cols + death_registry_cols if col not in cols_keep]
df_merged.drop(cols_to_drop, axis="columns", inplace=True)


# rename all fields as fieldID-visitID to be able to transform wide to long format
remaining_repeat_cols = [col for col in [col for col in df_merged.columns if "." in col] if col.split(".")[1]!="0"]
if remaining_repeat_cols:
    raise ValueError(f"Following columns have repeated measurements per visit, need to be aggregated: {remaining_repeat_cols}")
df_merged.rename(columns=lambda x: x.split(".")[0] if "." in x else x, inplace=True)


# Rename columns
static_cols_rename = {"31": "sex", "21000": "ethnicity", "189": "townsend", 
                      "200": "date_consent", 
                      "190": "reason_lfu", "191": "date_lfu", "40000": "date_death"}
df_merged.rename(columns=lambda x: static_cols_rename[x.split("-")[0]] 
                 if x.split("-")[0] in static_cols_rename.keys() else x, inplace=True)

variable_cols_rename = {"53": "visit_date", "54": "visit_centre",  
                        "age": "age",
                     "21001": "bmi", "102": "pulse", "4079": "dbp", "4080": "sbp",
                     "30000": "wbc", "30010": "rbc", "30020": "hgb", "30030": "hct", "30080": "plt", "30180": "lym", 
                     "30500": "u_alb", "30510": "u_cr", "30520": "u_potas", "30530": "u_sodium",
                     "30630": "apoa", "30640": "apob", "30670": "urea", "30690": "chol", "30710": "crp", "30720": "cysc", "30760": "hdl", "30770": "igf1", "30780": "ldl", 
                     "30830": "shbg", "30850": "tst", "30860": "tprotein", "30870": "tgly", "30890": "vitd",
                    }
df_merged.rename(columns=lambda x: "-".join([variable_cols_rename[x.split("-")[0]], x.split("-")[1]]) 
                 if x.split("-")[0] in variable_cols_rename.keys() else x, inplace=True)

label_cols_include = ['n_cancer_occurred', 'n_cancer_occurred_lastvisit', 'cancer_first_occurred_age', 'cancer_first_occurred_date',
                      'label_first_occurred_code', 'label_first_occurred_date', 'label_first_occurred_age',
                      'label_tumour_behaviour-0', 'label_tumour_behaviour-1', 'label_tumour_behaviour-2','label_tumour_behaviour-3',
                      'is_label-0', 'is_label-1', 'is_label-2','is_label-3','label_class']
# order columns
df_merged = df_merged[["eid"] + 
                      [col for col in df_merged.columns if col.startswith(tuple(static_cols_rename.values()))] + 
                      [col for col in df_merged.columns if col.startswith(tuple(variable_cols_rename.values()))] +
                      label_cols_include]

In [ ]:
# df_merged.to_csv("path/to/file_merged.csv", index=False)
df_merged = pd.read_csv("path/to/file_merged.csv")
print(df_merged.shape)

### Clean and process additional data release

In [ ]:
df2 = read_csv('path/to/file2.csv')
df2.head()

In [ ]:
df3 = read_csv("path/to/file3.csv")
pattern = re.compile(r"p(.+)_i(\d+)")
df3.rename(columns=lambda x: f"{pattern.search(x).group(1)}-{pattern.search(x).group(2)}.0" if "_i" in x else x, inplace=True)
df3["10004-0.0"] = df3["10004-0.0"].apply(lambda x: np.nan if pd.isnull(x) else eval(str(x)))
df3["10004-0.0"] = df3["10004-0.0"].apply(lambda x: np.nan if np.isnan(x).all() else (1 if 1 in x else np.nan)) # 1 if has aspirin else nan
df3[["6150-0.0", "6150-1.0", "6150-2.0"]] = df3[["6150-0.0", "6150-1.0", "6150-2.0"]].applymap(lambda x: np.nan if pd.isnull(x) else eval(str(x)))
df3[["20002-0.0", "20002-1.0", "20002-2.0", "20002-3.0"]] = df3[["20002-0.0", "20002-1.0", "20002-2.0", "20002-3.0"]].applymap(lambda x: np.nan if pd.isnull(x) else eval(str(x)))

df3.head()

In [ ]:
df2 = pd.merge(left=df2, right=df3, how="left", on="eid")

df2 = aggregate_columns(df2, field_ids=[23111, 23115], method="sum", drop_cols=True) # legfat pct left and right
df2 = aggregate_columns(df2, field_ids=[23119, 23123], method="sum", drop_cols=True) # armfat pct left and right
df2 = aggregate_columns(df2, field_ids=[46, 47], method="max", drop_cols=True) # hand grip left and right
df2 = compute_waist_to_hip_ratio(df2, drop_cols=False)
df2 = compute_trunk_to_leg_fat_ratio(df2, drop_cols=False)

df2 = mask_vals_as_nan_in_repeat_col(df2, field_id=20116, val=(-3)) # smoke -3 (Prefer not to answer) to np.nan
df2 = mask_vals_as_nan_in_repeat_col(df2, field_id=20117, val=(-3)) # alcohol -3 (Prefer not to answer) to np.nan
fields_na_vals_to_nan = [2345, 2178, 1349, 1369, 1379, 1389, 1329, 1289, 1299, 1309, 1319, 3456] # crc screening, health rating, intakes 
for field in fields_na_vals_to_nan:
    df2 = mask_vals_as_nan_in_repeat_col(df2, field_id=field, val=[-1,-3])
df2 = has_family_history(df2, disease="cancer", drop_cols=False)
df2 = has_family_history(df2, disease="colorectal_cancer", drop_cols=True)
df2 = has_disease_history_touchscreen(df2, disease="cardiovascular", list_format=True, drop_cols=True)
df2 = has_disease_history_touchscreen(df2, disease="diabetes", list_format=False, drop_cols=True)
df2 = has_disease_history_verbalinterview(df2, disease="ibd", list_format=True, drop_cols=False) 
df2 = has_disease_history_verbalinterview(df2, disease="anyliverbiliary", list_format=True, drop_cols=False) 
df2 = get_education_level(df2, level="university", drop_cols=True)
df2 = using_regular_medication(df2, medication="aspirin", drop_cols=False)
df2 = using_regular_medication_specific(df2, medication="statin", drop_cols=True)
df2 = calculate_redmeat_intake(df2, drop_cols=False)
df2 = is_fasted_before_assessment(df2, fast_min_hour=8, drop_cols=True)
df2 = combine_alcohol_status_and_frequency(df2)
df2 = combine_smoking_status_and_frequency(df2)

biomarker_repeat_cols_rename =  {
    "22333": "rr_ecg", 
    # 4 visits
    "48": "waist_cir", "49": "hip_cir", "whr": "whr", "tlr": "tlr",
    "23099": "bodyfat_pct", "23127": "trunkfat_pct", "23104": "bmi_imp", 
    "23111_23115_sum": "legfat_pct", "23119_23123_sum": "armfat_pct", "46_47_max": "hgrip",
    "23105": "met_rate", "23106": "impedance",
    "21021": "art_stiffness", "1160": "sleep_dur",
    # 3 visits
    "30040": "mcv", "30190": "mono", "30200": "neut", "30210": "eos", "30220": "baso", "30230": "n_rbc", "30240": "reti",
    # 2 visits
    "30600": "alb", "30610": "alp", "30620": "alt", "30650": "ast",  "30730": "ggt", "30660": "d_bil", "30840": "t_bil", 
    "30680": "calc","30740": "glu", "30750": "hba1c", "30790": "lipoa", "30800": "estradiol", "30810": "phos", 
    "30820": "rheum", "30880": "urate",
    "6033": "max_hr",
    # 1 visit
    "78": "heel_bmd_t", 
    "20256": "fev1_z", "20258": "fev1_fvc",
    "22040": "met_mins"
}

dem_cols_rename = {
    "50": "height",
    "20116": "smoke_status", 
    "20117": "alco_status", 
    "1558": "alco_freq",
    "3456": "n_cigarettes_daily",
    "1349": "processed_meat_intake",
    "1369": "beef_intake",
    "1379": "lamb_mutton_intake",
    "1389": "pork_intake",
    "1329": "oily_fish_intake",
    "1289": "cooked_veg_intake",
    "1299": "raw_veg_intake",
    "1309": "fresh_fruit_intake",
    "1319": "dried_fruit_intake",
    "2345": "crc_screening",
    "2178": "health_rating",
    "famhist_cancer": "famhist_cancer",
    "famhist_colorectal_cancer": "famhist_colorectal_cancer",
    "diseasehist_diabetes": "diseasehist_diabetes",
    "diseasehist_cardiovascular": "diseasehist_cardiovascular",
    "diseasehist_ibd": "diseasehist_ibd", 
    "diseasehist_anyliverbiliary": "diseasehist_anyliverbiliary", 
    "edu_university": "edu_university",
    "redmeat_intake": "redmeat_intake",
    "regular_aspirin": "regular_aspirin",
    "regular_statin": "regular_statin",
    "fasted": "fasted", 
    "alcohol":"alcohol", "smoke":"smoke"
}

df2.rename(columns=lambda x: "-".join([biomarker_repeat_cols_rename[x.split("-")[0]], x.split("-")[1]]) 
                 if x.split("-")[0] in biomarker_repeat_cols_rename.keys() else x, inplace=True)
df2.rename(columns=lambda x: "-".join([dem_cols_rename[x.split("-")[0]], x.split("-")[1]]) 
                 if x.split("-")[0] in dem_cols_rename.keys() else x, inplace=True)

df2 = df2[["eid"] + 
    [col for col in df2.columns if col.startswith(tuple(dem_cols_rename.values()))] + 
    [col for col in df2.columns if col.startswith(tuple(biomarker_repeat_cols_rename.values()))]
    ]

# rename all fields as fieldID-visitID to be able to transform wide to long format
remaining_repeat_cols = [col for col in [col for col in df2.columns if "." in col] if col.split(".")[1]!="0"]
if remaining_repeat_cols:
    raise ValueError(f"Following columns have repeated measurements per visit, need to be aggregated: {remaining_repeat_cols}")
df2.rename(columns=lambda x: x.split(".")[0] if "." in x else x, inplace=True)

In [ ]:
df_merged_new = pd.merge(left=df_merged, right=df2, how="left", on="eid")
print(df_merged_new.shape)
df_merged_new.head()

In [ ]:
df_merged_new.to_csv("path/to/file_merged_new.csv", index=False)

### Wide to long format

In [ ]:
df_merged = read_csv("path/to/file_merged_new.csv")

In [ ]:
unique_fields = set([col.split("-")[0] for col in df_merged.columns])
visit_per_field = {}
for field in unique_fields:
    n_visits = len([col for col in df_merged.columns if col.split("-")[0]==field])
    visit_per_field[field] = n_visits
sorted_visit_per_field = {k: v for k, v in sorted(visit_per_field.items(), key=lambda item: item[1])}

plt.figure(figsize=(12,10))
plt.rcParams.update({'font.size': 7})
plt.barh(*zip(*sorted_visit_per_field.items()))
plt.xticks(range(5))
plt.title("Number of visits available per data field")
plt.show()

In [ ]:
stub_cols_1visit = ["heel_bmd_t", "fev1_z", "fev1_fvc", "met_mins"]
stub_cols_at_least_2visits =  {field: n_visits for field, n_visits in sorted_visit_per_field.items() if n_visits>=2}
stub_cols_at_least_3visits =  {field: n_visits for field, n_visits in sorted_visit_per_field.items() if n_visits>=3}
stub_cols_at_least_4visits =  {field: n_visits for field, n_visits in sorted_visit_per_field.items() if n_visits>=4}
stub_cols_all = list(stub_cols_at_least_2visits.keys()) + stub_cols_1visit

In [ ]:
ids_at_least_1visits = df_merged[df_merged.filter(like="visit_date").notna().sum(axis=1) >= 1].eid.tolist()
ids_at_least_2visits = df_merged[df_merged.filter(like="visit_date").notna().sum(axis=1) >= 2].eid.tolist()
ids_at_least_3visits = df_merged[df_merged.filter(like="visit_date").notna().sum(axis=1) >= 3].eid.tolist()
ids_at_least_4visits = df_merged[df_merged.filter(like="visit_date").notna().sum(axis=1) >= 4].eid.tolist()
print(f"Number of participants with at least 1 visits: {len(ids_at_least_1visits)}")
print(f"Number of participants with at least 2 visits: {len(ids_at_least_2visits)}")
print(f"Number of participants with at least 3 visits: {len(ids_at_least_3visits)}")
print(f"Number of participants with at least 4 visits: {len(ids_at_least_4visits)}")

In [ ]:
df_long = pd.wide_to_long(df_merged, stub_cols_all, i='eid', j='visit', sep='-').reset_index()
df_long["is_label"] = df_long["is_label"].astype(float)
print(df_long.shape)

df_long = df_long[df_long['visit_date'].notna()] # remove visits which did not happen (i.e. null visit date) and so has no labs
df_long.to_csv("path/to/file_merged_new_longf.csv", index=False) 
print(df_long.shape)